In [2]:
import pandas as pd
import json
from sklearn.preprocessing import MinMaxScaler

In [8]:
df = pd.read_csv("combined_league_matches.csv")

with open("champion_max_winrates.json", "r") as file:
    win_rate_data = json.load(file)
win_rates = {int(k): float(v) for k, v in win_rate_data.items()}

blue_mastery_cols = [f'blue_mastery_{i}' for i in range(5)]
red_mastery_cols = [f'red_mastery_{i}' for i in range(5)]
blue_winrate_cols = [f'blue_champion_{i}' for i in range(5)]
red_winrate_cols = [f'red_champion_{i}' for i in range(5)]

scaler = MinMaxScaler(feature_range=(0.9, 1.1))

df[blue_mastery_cols + red_mastery_cols] = scaler.fit_transform(df[blue_mastery_cols + red_mastery_cols])

df['blue_score'] = sum(df[f'blue_champion_{i}'] * df[f'blue_mastery_{i}'] for i in range(5))
df['red_score'] = sum(df[f'red_champion_{i}'] * df[f'red_mastery_{i}'] for i in range(5))

df['predicted_winner'] = df.apply(lambda row: 'blue' if row['blue_score'] > row['red_score'] else 'red', axis=1)

accuracy = (df['winner'] == df['predicted_winner']).mean()
print(accuracy)

0.49365594570669813


In [3]:
df = pd.read_csv("combined_league_matches.csv")

with open("champion_max_winrates.json", "r") as file:
    win_rate_data = json.load(file)
win_rates = {int(k): float(v) for k, v in win_rate_data.items()}

# Define the columns for winrate
blue_winrate_cols = [f'blue_champion_{i}' for i in range(5)]
red_winrate_cols = [f'red_champion_{i}' for i in range(5)]

# Calculate total winrate for each team
df['blue_total_winrate'] = df[blue_winrate_cols].sum(axis=1)
df['red_total_winrate'] = df[red_winrate_cols].sum(axis=1)

# Predict winner based on higher total winrate
df['predicted_winner'] = df.apply(lambda row: 'blue' if row['blue_total_winrate'] > row['red_total_winrate'] else 'red', axis=1)

# Evaluate performance
accuracy = (df['winner'] == df['predicted_winner']).mean()
print(f'Accuracy: {accuracy:.2%}')

Accuracy: 49.34%
